In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import numpy as np
%config Completer.use_jedi = False

In [2]:
# Load the dataset
df = pd.read_table('lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', lineterminator='\n', warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
df_profile = pd.read_csv('lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=True, skiprows=1, names=['user', 'gender', 'age', 'country', 'signup'])

In [ ]:
df.head()

In [ ]:
# Check if the same user has the same song entered multiple times
df[['user', 'song', 'song-id']].groupby('user').agg({'count', 'nunique'})

In [ ]:
# Identify missing data percentage from each column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df

In [3]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [ ]:
# Check how many ids has more than 2 different song names
grouped[grouped['song'] > 2].shape[0]

In [ ]:
# Check how many ids has more than 1 song name
grouped[grouped['song'] > 1].shape[0]

In [ ]:
# Print examples of ids has more than 2 song names
grouped[grouped['song'] > 2]

In [ ]:
# Print examples of ids has 2 song names
grouped[grouped['song'] > 1]

In [ ]:
# Check an example of groups with more than 2 song names
df.loc[df['song-id'] == '4517e130-602c-4c63-b34f-cd6679eb0c46', 'song'].map(lambda x: x)

In [ ]:
# Check an example of groups with 2 song names
df.loc[df['song-id'] == 'f9e33eac-6977-4808-9edc-74f71a8de78a', 'song']

### Problems:

1. Inconsistency in the song names
1. Rows that have been appended to other rows

In [6]:
# Get rows with song names containing multiple rows
duplicated = df.loc[df['song-id'].isin(grouped[grouped['song'] > 1].index)]
duplicated = duplicated.drop(columns=['user', 'timestamp', 'artist', 'artist-id'])
containis_extra_rows = duplicated.apply(lambda x: pd.Series({'id': x[0], 'song': x[1], 'flag':'\n' in x[1]}), axis=1)
containis_extra_rows = containis_extra_rows.loc[containis_extra_rows['flag']]

In [7]:
# Iterate over those 5k rows to get the extra rows and correct the song name
for index, row in containis_extra_rows.iterrows():
    row_break = row['song'].split('\n', 1)
    song_name = row_break[0]
    df.loc[df['song-id'] == row['id'], 'song'] = song_name
    
    tsv = StringIO(row_break[1])
    df_extra = pd.read_csv(tsv, sep="\t", warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
    df = df.append(df_extra, ignore_index=True)

In [8]:
# Now we save the full data as a check point
df.to_parquet('lastfm-dataset-1K/df_full.parquet.gzip', compression='gzip')

In [3]:
df = pd.read_parquet('lastfm-dataset-1K/df_full.parquet.gzip')

In [4]:
# Adding the new rows we recount the missing values
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
user,user,0.000000
timestamp,timestamp,0.000000
artist-id,artist-id,3.145175
artist,artist,0.000000
song-id,song-id,11.324443
song,song,0.000073


In [6]:
df = df.drop_duplicates()

### Fix names

In [ ]:
df_names = df.groupby('song-id').head(1).dropna()
df = df.drop(columns='song').merge(df_names, on='song-id', how='left')

### Create matrix

In [ ]:
df_play_count = df.groupby(['user', 'song']).count()

### Filling NA values:
We will check if the songs corresponding to missing ids exist somewhere else

In [ ]:
df.head()

In [ ]:
df_songs_names = df[['artist', 'song', 'song-id']].groupby(['artist', 'song-id']).head(1)
df_songs_names = df_songs_names.dropna().set_index(['artist', 'song'])

In [ ]:
df_songs_names.head()

In [ ]:
def map_to_song_id(row):
    return df_songs_names.loc[row[3], row[5]] if (row[3], row[5]) in df_songs_names.index else np.nan

In [ ]:
df.loc[df['song-id'].isnull(), 'song-id'] = df.loc[df['song-id'].isnull()].apply(map_to_song_id, axis=1)

In [ ]:
# Adding the new rows we recount the missing values
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
songs_missing_id = df.loc[df['song-id'].isnull(), 'song']

In [ ]:
songs_missing_id_set = set(songs_missing_id)
songs_map = set(df_songs_names.index)

In [ ]:
songs_missing_id_set & songs_map

In [ ]:
df[df['song'] == 'Hibari (Live_2009_4_15)']

In [ ]:
df_songs_names.loc['Who Told You']

In [ ]:
grouped_songs = df[['song-id', 'song']].groupby(['song']).nunique()

In [ ]:
# Check how many ids has more than 2 different song names
grouped_songs[grouped_songs['song-id'] > 3].shape[0]

In [ ]:
# look at some examples of same song names with different ids
grouped_songs[grouped_songs['song-id'] > 3]

In [ ]:
df.loc[df['song'] == 'Alvorada']

In [ ]:
df.loc[df['song-id'] == 'bc07ec39-8f80-4d33-8cbe-fc575d459fbf', 'song']

In [ ]:
df.loc[df['song-id'].isnull()]

In [ ]:
k = df.loc[df['song-id'].isnull()].loc[19098818].to_numpy().tolist()

In [ ]:
k[3], k[5]

In [ ]:
df_songs_names[k[3], k[5]]

In [ ]:
df_songs_names.index